In [24]:
!pip install evaluate

In [1]:
# from transformers import pipeline
# from sklearn.metrics import accuracy_score

In [4]:
# sentiment_analysis = pipeline(
#     'sentiment-analysis', 
#      model="distilbert-base-uncased-finetuned-sst-2-english",
#     revision="714eb0f"   # optional but ensures reproducibility
# )

# test_examples = [
#     {"text": "I love this product!", "label": 1}, 
#     {"text": "The service was terrible.", "label": 0}, 
#     {"text": "This movie is amazing.", "label": 1}, 
#     {"text": "I'm disappointed with the quality.", "label": 0}
# ]

# predictions = sentiment_analysis(
#     [example["text"] for example in test_examples] 
# )

In [2]:
import evaluate

In [2]:
accuracy = evaluate.load("accuracy")
print(accuracy.description)


Accuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
 Where:
TP: True positive
TN: True negative
FP: False positive
FN: False negative



In [3]:
print(evaluate.load("f1").description)


The F1 score is the harmonic mean of the precision and recall. It can be computed with the equation:
F1 = 2 * (precision * recall) / (precision + recall)



In [4]:
print(accuracy.features)

{'predictions': Value('int32'), 'references': Value('int32')}


In [5]:
f1 = evaluate.load("f1")
print(f1.features)

{'predictions': Value('int32'), 'references': Value('int32')}


In [6]:
pearson_scorr = evaluate.load("pearsonr")
print(pearson_scorr.features)

{'predictions': Value('float32'), 'references': Value('float32')}


In [7]:
# !pip install transformers torch

In [8]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch

# 1. Load tokenizer & model
# ---------------------------
model_name = "gpt2"

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# GPT-2 does not have a padding token → set pad token to eos token to avoid warnings
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [9]:
# model_name = "gpt2"

prompt = "Latest research findings in Antarctica show"

prompt_ids = tokenizer.encode(prompt, return_tensors = 'pt')  # test_sentence
output = model.generate(prompt_ids, max_length = 17)
generated_text = tokenizer.decode(
    output[0], skip_special_tokens = True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Latest research findings in Antarctica show that the ice sheet is melting faster than previously thought.


In [10]:
perplexity = evaluate.load('perplexity', module_type = "metric")
results = perplexity.compute(model_id = 'gpt2', predictions = generated_text)
print(results['mean_perplexity'])

  0%|          | 0/7 [00:00<?, ?it/s]

2867.6824600219725


In [11]:
perplexity.features

{'predictions': Value('string')}

In [13]:
!pip install rouge_score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=3e940bf59c90559b6460da427eaf05552c733b27aaaa9389e7bf5300194c1922
  Stored in directory: c:\users\dr. bhavesh dharmani\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [1]:
import evaluate

rouge = evaluate.load("rouge")

predictions = ["""As we learn more about the frequency and size distribution of exoplanets, we are dicovering 
    that terrestrial planets are exceedingly common."""]

references = ["""The more we learn about the frequency and size distribution of exoplanets, the more
confident we are that they are exceedingly common."""]
results = rouge.compute(predictions = predictions, references = references)
print(results)

{'rouge1': 0.7906976744186046, 'rouge2': 0.5365853658536585, 'rougeL': 0.7441860465116279, 'rougeLsum': 0.7441860465116279}


In [2]:
print(results["rouge1"])

0.7906976744186046


In [3]:
rouge2 = evaluate.load("rouge")
results1 = rouge.compute(predictions = predictions, references = references)
print(results1)

{'rouge1': 0.7906976744186046, 'rouge2': 0.5365853658536585, 'rougeL': 0.7441860465116279, 'rougeLsum': 0.7441860465116279}


In [4]:
# Rouge clipping implmentation 

# import evaluate
# rouge = evaluate.load("rouge")

# Reference summary
reference = ["The cat sat on the mat"]

# Candidate with repetition (trying to game ROUGE)
predictions = ["The the the the the"]

# Candidate with correct wording
candidate_correct = ["The cat sat on the mat"]

# Compute ROUGE
results_repeated = rouge.compute(predictions=predictions, references=reference)
results_correct = rouge.compute(predictions=candidate_correct, references=reference)

print("Repeated candidate:", results_repeated)
print("Correct candidate:", results_correct)

Repeated candidate: {'rouge1': 0.3636363636363636, 'rouge2': 0.0, 'rougeL': 0.3636363636363636, 'rougeLsum': 0.3636363636363636}
Correct candidate: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


In [6]:
print(rouge.features)

[{'predictions': Value('string'), 'references': List(Value('string'))}, {'predictions': Value('string'), 'references': Value('string')}]


In [9]:
bleu = evaluate.load("bleu")

references = ["I am very happy to say that I am drinking a warm cup of tea."]

predictions_1 = ["I am very happy that I am drinking a cup of tea."]
predictions_2 = ["I am very happy that I am drinking a warm cup of tea."]
predictions_3 = ["I am very happy to say that I am drinking a cup of tea."] 
predictions_4 = ["I am very happy to say that I am drinking a warm cup of tea."]

results_1 = bleu.compute(predictions = predictions_1, references = references)
results_2 = bleu.compute(predictions = predictions_2, references = references)
results_3 = bleu.compute(predictions = predictions_3, references = references)
results_4 = bleu.compute(predictions = predictions_4, references = references)
print(results_1)
print(results_2)
print(results_3)
print(results_4)

{'bleu': 0.5387958246253988, 'precisions': [1.0, 0.8333333333333334, 0.6363636363636364, 0.4], 'brevity_penalty': 0.7939226578179512, 'length_ratio': 0.8125, 'translation_length': 13, 'reference_length': 16}
{'bleu': 0.7497153770440844, 'precisions': [1.0, 0.9230769230769231, 0.8333333333333334, 0.7272727272727273], 'brevity_penalty': 0.8668778997501817, 'length_ratio': 0.875, 'translation_length': 14, 'reference_length': 16}
{'bleu': 0.8196501312471536, 'precisions': [1.0, 0.9285714285714286, 0.8461538461538461, 0.75], 'brevity_penalty': 0.9355069850316178, 'length_ratio': 0.9375, 'translation_length': 15, 'reference_length': 16}
{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 16, 'reference_length': 16}


In [5]:
import evaluate

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

preds = ["""He thought it right and necessary to become a knight-errant, travelling the 
world in armor, 
seekng adventures and practicing the deeds he had read about in chivalric tales."""]

refs = [""" He believed it was proper and essential to transform into knight-errant, 
travelling the world in armor, persuing adventures, amd enacting the heeroic deeds he
had encountered in tales of chivalry."""]

results_b = bleu.compute(predictions = preds, references = refs)
results_m = meteor.compute(predictions = preds, references = refs)
print("Bleu: ", results_b)
print("Meteor: ", results_m)

[nltk_data] Downloading package wordnet to C:\Users\Dr. Bhavesh
[nltk_data]     Dharmani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Dr. Bhavesh
[nltk_data]     Dharmani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Dr. Bhavesh
[nltk_data]     Dharmani\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Bleu:  {'bleu': 0.27434490219813984, 'precisions': [0.6451612903225806, 0.3, 0.2413793103448276, 0.17857142857142858], 'brevity_penalty': 0.9077609612738833, 'length_ratio': 0.9117647058823529, 'translation_length': 31, 'reference_length': 34}
Meteor:  {'meteor': 0.5095984981529704}


In [7]:
import evaluate 

em = evaluate.load("exact_match")
predictions = ["The cat sat on the mat.", 
            "Theaters are great.", 
            "It's like comparing oranges and apples."]
references = ["The cat sat on the mat?", 
             "Theaters are great.", 
             "It's like comparing apples and oranges."]
results_em = em.compute(predictions = predictions, references = references)
print(results_em)

{'exact_match': 0.3333333333333333}


In [12]:
import evaluate

toxicity = evaluate.load("toxicity")

pred_1 = ["Everyone in the team adores him.",
         "He is a true genius, pure talent."]
pred_2 = ["Nobody in the team likes him.", 
         "He is a useless 'good-for-nothing'. "]

toxicity_res1 = toxicity.compute(predictions = pred_1)
print("Toxicity in Pred_1 is: ", toxicity_res1)
toxicity_res1 = toxicity.compute(predictions = pred_2)
print("Toxicity in Pred_2 is: ", toxicity_res1)

Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint
Device set to use cpu


Toxicity in Pred_1 is:  {'toxicity': [0.0001413044665241614, 0.00013581576058641076]}
Toxicity in Pred_2 is:  {'toxicity': [0.00014405461843125522, 0.0057794381864368916]}


In [13]:
toxicity_res1 = toxicity.compute(predictions = pred_1, aggregation = "maximum")
print("Toxicity in Pred_1 is: ", toxicity_res1)
toxicity_res1 = toxicity.compute(predictions = pred_2, aggregation = "maximum")
print("Toxicity in Pred_2 is: ", toxicity_res1)

Toxicity in Pred_1 is:  {'max_toxicity': 0.0001413044665241614}
Toxicity in Pred_2 is:  {'max_toxicity': 0.0057794381864368916}


In [14]:
toxicity_res1 = toxicity.compute(predictions = pred_1, aggregation = "ratio")
print("Toxicity in Pred_1 is: ", toxicity_res1)
toxicity_res1 = toxicity.compute(predictions = pred_2, aggregation = "ratio")
print("Toxicity in Pred_2 is: ", toxicity_res1)

Toxicity in Pred_1 is:  {'toxicity_ratio': 0.0}
Toxicity in Pred_2 is:  {'toxicity_ratio': 0.0}


In [15]:
import evaluate

regard = evaluate.load("regard")

pred_1 = ["abc are described as loyal employees.", 
         "abc are ambitious in their career expectations."]
pred_2 = ["abc are known for causing lots of team conflicts.", 
         "abc are verbally violent."]

eval_1 = regard.compute(data = pred_1)
eval_2 = regard.compute(data = pred_2)
print("eval_1: ", eval_1)
print("eval_2: ", eval_2)

Device set to use cpu


eval_1:  {'regard': [[{'label': 'positive', 'score': 0.8298447728157043}, {'label': 'other', 'score': 0.0996326208114624}, {'label': 'neutral', 'score': 0.05163787677884102}, {'label': 'negative', 'score': 0.018884729593992233}], [{'label': 'positive', 'score': 0.3437841236591339}, {'label': 'neutral', 'score': 0.3377510905265808}, {'label': 'other', 'score': 0.18729078769683838}, {'label': 'negative', 'score': 0.13117395341396332}]]}
eval_2:  {'regard': [[{'label': 'negative', 'score': 0.9624267816543579}, {'label': 'other', 'score': 0.023236598819494247}, {'label': 'neutral', 'score': 0.013599522411823273}, {'label': 'positive', 'score': 0.0007369781378656626}], [{'label': 'negative', 'score': 0.9755293130874634}, {'label': 'other', 'score': 0.014527562074363232}, {'label': 'neutral', 'score': 0.009333696216344833}, {'label': 'positive', 'score': 0.0006093884003348649}]]}
